## Qn

Given a string s, return the longest palindromic substring in s.

- Example 1:
    - Input: s = "babad"
    - Output: "bab"
    - Explanation: "aba" is also a valid answer.

- Example 2:
    - Input: s = "cbbd"
    - Output: "bb"
 
- Constraints:
    - 1 <= s.length <= 1000
    - s consist of only digits and English letters.

## Option 1: Brute Force

Let's start by discussing a brute force strategy. To use brute force, you simply exhaustively explore all possible strings, and check if they are palindromes.

For a given string S of length N, there are N choose 2 ways to get a pair of start and end points. This evaluates to $\frac {N!} {2! \cdot (N-2)!}$, which simplifies to $\frac {N \cdot (N-1)} {2}$. In Big O notation, this is $O(N^2)$.

But of course, evaluating if each string is a palindrome also takes effort. The best way to check if a string is a palindrome is to start from the middle, and expand outwards. In the worst case, where all chosen substrings are palindromes, the check takes $O(M)$ where M is the length of the substring (i.e. each check will iterate over the entire length of the substring). Since we are comparing all substrings, $M = \frac{N}{2}$. Hence, the check itself will take $O(N/2)$ or $O(N)$ time.

This approach has time complexity of $O(N^3)$, and space complexity of $O(1)$.

In [27]:
def check_palindrome(s: str) -> bool:
    if len(s) == 0 or len(s) == 1:
        return True

    # For even length strings, add a character to it's middle to evaluate palindrome
    if len(s) % 2 == 0:
        s = s[:len(s)//2] + '~' + s[(len(s)//2):]
    # Starting from the middle character, expand outwards 
    left=right=len(s)//2
    while s[left]==s[right]:
        left-=1
        right+=1
        if left < 0:
            return True
    return False

def longest_palindromic_substring_bruteforce(s: str) -> str:
    left=right=0
    longest_palindromic_substring = ''
    for left in range(len(s)):
        for right in range(left, len(s)):
            substring_length = len(s[left:(right+1)])
            is_palindrome = check_palindrome(s[left:(right+1)])
            
            if is_palindrome & (substring_length > len(longest_palindromic_substring)):
                longest_palindromic_substring = s[left:(right+1)]
    return longest_palindromic_substring

longest_palindromic_substring_bruteforce('wow')

'wow'

## Option 2: Dynamic Programming

Let's observe palindromes more closely. Suppose that we have some known palindromic string `X`. If we add characters to either side of `X`, we will make string `aXa` if both characters are the same, or `aXb` if they are different. In the first situation, `aXa` will also be a palindrome.

This means that we do not need to expend additional resources evaluating the encapsulated string X once more, which suggests that every substring is only evaluated once! This turns the previous time complexity of $O(N^3)$ to $O(N^2)$ instead, but the space complexity goes from $O(1)$ to $O(N^2)$, because you store $N^2$ values in the dp table.

As with all DP solutions, we will build up from shorter to longer strings and store our intermediate solutions which we then refer to when encountering longer strings.

In [41]:
def longest_palindromic_substring_dp(s: str) -> str:
    ## Initialise "table" to store values
    dp = [[False]*len(s) for x in range(len(s))]

    ## Longest palindrome is first letter by default (let's just define 1 letter as a palindrome)
    longest_palindrome = s[0]

    ## Build from length 1 strings to full length N string...
    for string_len in range(1, len(s)+1):
        ## Iterate through every viable starting index for strings of length string_len. 
        ## e.g. if the full string is 5 characters long, and you are looking for all possible substrings of length 3, it doesn't
        ## make sense to look beyond the element at index 2 (012, 123, 234), because there are no strings with length 3 beyond it
        for string_start_index in range(len(s)-string_len+1):

            ## If string length is 1, assign True to the relevant dp object
            if string_len == 1:
                dp[string_start_index][string_start_index]=True

            ## If string length is 2, assign True to the relevant dp object if both the characters match
            elif (string_len == 2) & (s[string_start_index] == s[string_start_index+string_len-1]):
                dp[string_start_index][string_start_index+string_len-1] = True
                if len(longest_palindrome) < string_len:
                    longest_palindrome = s[string_start_index:(string_start_index+string_len)]

            ## If string length exceeds 2, assign True to the relevant dp object if both the characters match AND the middle substring is palindrome.
            ## This can be looked up from the table, because we already computed all shorter substrings
            elif (string_len > 2) & (s[string_start_index] == s[string_start_index+string_len-1]) & (dp[string_start_index+1][string_start_index+string_len-2]):
                dp[string_start_index][string_start_index+string_len-1] = True
                if len(longest_palindrome) < string_len:
                    longest_palindrome = s[string_start_index:(string_start_index+string_len)]
    return longest_palindrome

longest_palindromic_substring_dp('wtfftw')

'wtfftw'

## Option 3: Expand from center 

The obvious downside to the DP solution above is, of course, that you actually need to maintain a table of "answers" for shorter substrings, which takes up unnecessary space. In fact, this isn't strictly necessary. Notice that all palindromes must be symmetric about the center. Hence, to find the longest palindrome, we iterate through each character in the string and find the longest possible palindrome with it as the "center". 

To be more precise, we will visit each character, and start comparing the characters to its left and right, and proceed only if they match. This will help to avoid unnecessary palindrome checks. Since we are visitng N possible characters, and in the worse case we expand each center by N/2 steps, the time complexity of this algorithm is $O(N^2)$. But since we are doing away with the dp table, the space complexity is simply $O(1)$.

One interesting note here. Since we are no longer iteratively exploring strings, from left to right, but instead expanding them from potential centers, we will run into issues when we encounter even numbered palindromes. For example, `aba` is obviously centered around `b`, but `abba` has no character center point. To get around this issue, we will put in special characters between each character which allows us to consider this expansion.

In [43]:
def longest_palindromic_substring_expandfromcenter(s: str) -> str:
    
    ## Add character '~' in between every character in the string to account for even length palindromes
    s_mod = list(s)
    s_mod = ''.join([''.join(x) for x in zip(['~' for _  in range(len(s_mod))], s_mod)]) + '~'
    
    ## Set lps to the first character of the string, in the absence of anything longer we define singletons are palindromes
    lps = s_mod[0]

    ## Loop through every character in the string (including the new ~ character) as a "center" candidate
    ## We start from 1 and end of the second last character, because obviously you cannothave a palindrome centered at the start and end
    for center in range(1,len(s_mod)-1):
        distance_from_center=0

        # Keep expanding distance_from_center while...
        while (
            ((center - distance_from_center) >= 0) & ##... you have additional characters on the left 
            ((center + distance_from_center) <= (len(s_mod))) & ##... and you have additional characters on the right
            (s_mod[center-distance_from_center]==s_mod[center+distance_from_center]) ##... and the characters at current expansion from center match
        ):
            if (
                ((center-distance_from_center) <= 0) or ## Unless you are at the left boundary
                ((center+distance_from_center) >= len(s_mod)-1) or ## or you are at the right boundary
                (s_mod[center-distance_from_center-1]!=s_mod[center+distance_from_center+1]) ## or the next characters don't match
            ):
                break
        
            distance_from_center+=1 #expand 1 more character left and right

            #If the length of the current expansion exceeds the last known LPS, store it as the LPS
            if len(s_mod[(center-distance_from_center):(center+distance_from_center)]) > len(lps):
                lps = s_mod[(center-distance_from_center):(center+distance_from_center+1)]
    
    #Return LPS without the special characters ~
    return lps.replace('~','')

longest_palindromic_substring_expandfromcenter('abcde')

'a'

## Option 4: Manacher's Algorithm

Our best attempt so far has been $O(N^2)$ time and $O(1)$ space complexity. But amazingly, there is in fact an $O(N)$ time complexity solution!

The idea here is to expand on the solution in option 3, but we further eliminate needless expansions from center. Let's consider a string `abaaba`. For each of reference, I will denote each character with a unique subscript identifier, but note that this has no bearing on the palindrome comparison.

$a_1b_2a_3a_4b_5a_6$

Based on the previous solution, we will go from left to right, considering each character and space as a potential center candidate.

Let's pause at this character in our loop: $a_1b_2a_3<a_4>b_5a_6$. At this point, what do we already know? 
- We know for a fact that there is symmetry about the mid point around the first and second `aba`
- We also know that there is no palindrome around $a_3$
- But by the preceding 2 facts, it must also be the case that there is no palindrome around $a_4$
    - If there was one, then $a_1$ to $a_3$ cannot possibly be symmetric with $a_4$ to $a_6$
- This makes the palindrome check for $a_4$ unnecessary!

As such, the idea behind Manacher's algorithm is to give up some space $O(N)$ to record what we see in each of the earlier centers, and through this, we avoid expending compute on substrings that we know are definitely not palindromes, or substrings that we already know are palindromes. By doing this, it helps us reduce our time complexity to $O(N)$. Why $O(N)$?

In [ ]:
def longest_palindromic_substring_manacher(s: str) -> str:
    

## Option 5: Suffix Tree / Eertree

Another O(N) solution is to build a suffix tree for the given string

## Answer

In [4]:
def longestPalindrome(s: str) -> str:
    n = len(s)
    
    def getLen(l, r): 
        while l >= 0 and r < n and s[l] == s[r]:
            l -= 1
            r += 1
        return r - l -1

    start = 0
    length = 0
    for i in range(n):
        cur = max(
            getLen(i,i),
            getLen(i,i+1)
        )
        if cur <= length: 
            continue
        length = cur
        start = i - (cur - 1) // 2
    return s[start: start+length]

longestPalindrome('wtfftw')

'wtfftw'

In [8]:
def findLongestPalindromicString(text):
    N = len(text)
    print(f'text || {text}')
    print(f'len(text) || {len(text)}')

    if N == 0:
        return
    N = 2*N+1    # Position count
    print(f'Count positions in text 2*N+1 || {2*N+1}')

    L = [0] * N
    L[0] = 0
    L[1] = 1
    C = 1     # centerPosition
    R = 2     # centerRightPosition
    i = 0    # currentRightPosition
    iMirror = 0     # currentLeftPosition
    maxLPSLength = 0
    maxLPSCenterPosition = 0
    start = -1
    end = -1
    diff = -1
   
    # Uncomment it to print LPS Length array
    # printf("%d %d ", L[0], L[1]);
    for i in range(2,N):
        print('='*50)
       
        # get currentLeftPosition iMirror for currentRightPosition i
        iMirror = 2*C-i
        L[i] = 0
        diff = R - i

        print(f"i, C, iMirror, diff || {i}, {C}, {iMirror}, {diff}")
        # If currentRightPosition i is within centerRightPosition R
        if diff > 0:
            print(f'diff > 0 || {diff} > 0')
            print(f'min(L[iMirror], diff) || min({L[iMirror]}, {diff})')
            L[i] = min(L[iMirror], diff)
   
        # Attempt to expand palindrome centered at currentRightPosition i
        # Here for odd positions, we compare characters and
        # if match then increment LPS Length by ONE
        # If even position, we just increment LPS by ONE without
        # any character comparison
        try:
            print(f'(i + L[i]) < N || ({i} + {L[i]}) < {N}')
            print(f'((i - L[i]) > 0 || ({i} - {L[i]}) > 0')
            print(f'((i + L[i] + 1) % 2 == 0) || (({i} + {L[i]} + 1) % 2 == 0)')
            print(f'(text[(i + L[i] + 1) // 2] == text[(i - L[i] - 1) // 2]) || \
                  (text[({i} + {L[i]} + 1) // 2] == text[({i} - {L[i]} - 1) // 2]) \
                  ({text[(i + L[i] + 1) // 2]} == {text[(i - L[i] - 1) // 2]}) ')
            while ((i + L[i]) < N and (i - L[i]) > 0) and \
                    (((i + L[i] + 1) % 2 == 0) or \
                    (text[(i + L[i] + 1) // 2] == text[(i - L[i] - 1) // 2])):
                L[i]+=1
        except Exception as e:
            pass
        
        if L[i] > maxLPSLength:        # Track maxLPSLength
            print(f'L[i] > maxLPSLength ||  {L[i]} > {maxLPSLength}')
            maxLPSLength = L[i]
            maxLPSCenterPosition = i
   
        # If palindrome centered at currentRightPosition i
        # expand beyond centerRightPosition R,
        # adjust centerPosition C based on expanded palindrome.
        if i + L[i] > R:
            print(f'i + L[i] > R ||  {i}+ {L[i]} > {R}')
            C = i
            R = i + L[i]
   
    # Uncomment it to print LPS Length array
    # printf("%d ", L[i]);
    start = (maxLPSCenterPosition - maxLPSLength) // 2
    end = start + maxLPSLength - 1
    print ("LP S of string is " + text + " : ",text[start:end+1])

In [9]:
findLongestPalindromicString('abcbab')

text || abcbab
len(text) || 6
Count positions in text 2*N+1 || 27
i, C, iMirror, diff || 2, 1, 0, 0
(i + L[i]) < N || (2 + 0) < 13
((i - L[i]) > 0 || (2 - 0) > 0
((i + L[i] + 1) % 2 == 0) || ((2 + 0 + 1) % 2 == 0)
(text[(i + L[i] + 1) // 2] == text[(i - L[i] - 1) // 2]) ||                   (text[(2 + 0 + 1) // 2] == text[(2 - 0 - 1) // 2])                   (b == a) 
i, C, iMirror, diff || 3, 1, -1, -1
(i + L[i]) < N || (3 + 0) < 13
((i - L[i]) > 0 || (3 - 0) > 0
((i + L[i] + 1) % 2 == 0) || ((3 + 0 + 1) % 2 == 0)
(text[(i + L[i] + 1) // 2] == text[(i - L[i] - 1) // 2]) ||                   (text[(3 + 0 + 1) // 2] == text[(3 - 0 - 1) // 2])                   (c == b) 
L[i] > maxLPSLength ||  1 > 0
i + L[i] > R ||  3+ 1 > 2
i, C, iMirror, diff || 4, 3, 2, 0
(i + L[i]) < N || (4 + 0) < 13
((i - L[i]) > 0 || (4 - 0) > 0
((i + L[i] + 1) % 2 == 0) || ((4 + 0 + 1) % 2 == 0)
(text[(i + L[i] + 1) // 2] == text[(i - L[i] - 1) // 2]) ||                   (text[(4 + 0 + 1) // 2] == text[(4 - 0 -